In [ ]:
!pip install -q tqdm>=4.66.4 rdkit>=2024.3.5

In [ ]:
!pip install -q torch-optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.3 MB/s eta 0:00:00


In [ ]:
!pip -q install git+https://github.com/idiap/fast-transformers.git

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q -U numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 42.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.1 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.1 which is incompatible.


In [ ]:
! git clone https://github.com/IBM/materials.git
! mv /content/materials/models/smi_ted/smi_ted_light /content/

In [ ]:
import sys
sys.path.append('../inference')

In [ ]:
# materials.smi-ted (smi-ted)
from smi_ted_light.load import load_smi_ted

# Data
import pandas as pd
import numpy as np
import torch

# Chemistry
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.DataStructs import FingerprintSimilarity
from rdkit.DataStructs import TanimotoSimilarity

In [ ]:
# function to canonicalize SMILES
def normalize_smiles(smi, canonical=True, isomeric=False):
    try:
        normalized = Chem.MolToSmiles(
            Chem.MolFromSmiles(smi), canonical=canonical, isomericSmiles=isomeric
        )
    except:
        normalized = None
    return normalized

# function to calculate pairwise Tanimoto similarity
def calculate_tanimoto_similarities(fps1, fps2):
    similarities = []
    for i in range(len(fps1)):
            sim = TanimotoSimilarity(fps1[i], fps2[i])
            similarities.append(sim)
    return similarities

In [ ]:
model_smi_ted = load_smi_ted(
    folder='../inference/smi_ted_light',
    ckpt_filename='smi-ted-Light_40.pt'
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


bert_vocab_curated.txt: 0.00B [00:00, ?B/s]

smi-ted-Light_40.pt:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

Random Seed: 12345
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Vocab size: 2393
[INFERENCE MODE - smi-ted-Light]


In [ ]:
df_moses = pd.read_csv("/content/materials/models/smi_ted/notebooks/data/moses_test.csv", nrows=1000)

In [ ]:
df_moses['SMILES'] = df_moses['SMILES'].apply(normalize_smiles)
df_test_normalized = df_moses.dropna()
print(df_test_normalized.shape)
df_test_normalized.head()

(1000, 1)


,SMILES
0,CC1C2CCC(C2)C1CN(CCO)C(=O)c1ccc(Cl)cc1
1,COc1ccc(-c2cc(=O)c3c(O)c(OC)c(OC)cc3o2)cc1O
2,CCOC(=O)c1ncn2c1CN(C)C(=O)c1cc(F)ccc1-2
3,Clc1ccccc1-c1nc(-c2ccncc2)no1
4,CC(C)(Oc1ccc(Cl)cc1)C(=O)OCc1cccc(CO)n1


In [ ]:
with torch.no_grad():
    encode_embeddings = model_smi_ted.encode(df_moses['SMILES'], return_torch=True)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 10/10 [00:03<00:00,  2.86it/s]


In [ ]:
with torch.no_grad():
    decoded_smiles = model_smi_ted.decode(encode_embeddings)

In [ ]:
encode_embeddings

tensor([[ 0.3554, -0.5505,  0.0285,  ...,  0.7829,  0.6323, -0.0801],
        [ 0.5972, -0.5081,  0.1212,  ...,  0.7434,  0.5760, -0.0281],
        [ 0.4411, -0.5010,  0.0364,  ...,  0.7492,  0.5562, -0.1500],
        ...,
        [ 0.3681, -0.5893,  0.0932,  ...,  0.8411,  0.5857, -0.1708],
        [ 0.2937, -0.4848,  0.1228,  ...,  0.8431,  0.6063, -0.1008],
        [ 0.4474, -0.4788,  0.0835,  ...,  0.8390,  0.5940, -0.1041]])